In [1]:
# pip install nltk 
# from the pop up window install: punkt, Averaged perceptron Tagger
# nltk.download()

In [2]:
#pip install spacy

In [3]:
import json
import nltk
from nltk.tag import StanfordNERTagger
import os
import spacy
import textacy
import pandas as pd

In [4]:
tickers= pd.read_json('/Users/macbookpro/Documents/GitHub/nlp/data/word_vectors/SP500_Tickers.json', typ='series')


url2='/Users/macbookpro/Documents/GitHub/nlp/data/word_vectors/article_headers_with_dates.txt'
header = pd.read_csv(url2,sep=",")
    


In [5]:
ticker = tickers.to_frame('Ticker').reset_index()
ticker.columns=['Name','Ticker']
ticker

,Name,Ticker
0,AGILENT TECHNOLOGIES,A
1,AMERICAN AIRLINES,AAL
2,ADVANCE AUTO PARTS,AAP
3,APPLE,AAPL
4,ABBVIE,ABBV
...,...,...
495,XYLEM,XYL
496,YUM! BRANDS,YUM
497,ZIMMER BIOMET HOLDINGS,ZBH
498,ZIONS BANCORPORATION NA,ZION


In [6]:
isin = pd.DataFrame(ticker['Name'].str.contains('APPLE|XYLEM', regex=True))
isin
index = isin[isin['Name']==True].reset_index()['index']
ticker.loc[index,'Ticker']

3      AAPL
495     XYL
Name: Ticker, dtype: object

In [7]:
list_ticker = ticker['Name'].to_list()
str1 = '|'.join(list_ticker)
str2 = str1.replace(' ','|')
str2

"AGILENT|TECHNOLOGIES|AMERICAN|AIRLINES|ADVANCE|AUTO|PARTS|APPLE|ABBVIE|AMERISOURCEBERGEN|ABIOMED|ABBOTT|LABORATORIES|ACCENTURE|ADOBE|ANALOG|DEVICES|ARCHER-DANIELS-MIDLAND|AUTOMATIC|DATA|PROCESSING|ALLIANCE|DATA|SYSTEMS|AUTODESK|AMEREN|AMERICAN|ELECTRIC|POWER|AES|AFLAC|ALLERGAN|AMERICAN|INTERNATIONAL|APARTMENT|INVESTMENT|&|MANAGEMENT|ASSURANT|ARTHUR|J|GALLAGHER|&|AKAMAI|TECHNOLOGIES|ALBEMARLE|ALIGN|TECHNOLOGY|ALASKA|AIR|ALLSTATE|ALLEGION|ALEXION|PHARMACEUTICALS|APPLIED|MATERIALS|AMCOR|ADVANCED|MICRO|DEVICES|AMETEK|AFFILIATED|MANAGERS|AMGEN|AMERIPRISE|FINANCIAL|AMERICAN|TOWER|AMAZONCOM|ARISTA|NETWORKS|ANSYS|ANTHEM|AON|A|O|SMITH|APACHE|AIR|PRODUCTS|&|CHEMICALS|AMPHENOL|APTIV|ALEXANDRIA|REAL|ESTATE|EQUITIES|ARCONIC|ATMOS|ENERGY|ACTIVISION|BLIZZARD|AVALONBAY|COMMUNITIES|BROADCOM|AVERY|DENNISON|AMERICAN|WATER|WORKS|AMERICAN|EXPRESS|AUTOZONE|BOEING|BANK|OF|AMERICA|BAXTER|INTERNATIONAL|BB&T|BEST|BUY|BECTON|DICKINSON|&|FRANKLIN|RESOURCES|BROWN-FORMAN|BAKER|HUGHES|A|GE|BIOGEN|BANK|OF|NEW|YORK|M

In [8]:
isin = pd.DataFrame(header['Header'].str.contains(str1, regex=True))
isin

/Users/macbookpro/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,Header
0,False
1,False
2,False
3,False
4,False
...,...
47846,False
47847,False
47848,False
47849,False


In [9]:
index = isin[isin['Header']==True].reset_index()['index']
#ticker.loc[index,'Ticker']
#ticker
index

0        486
1        625
2        690
3        756
4        897
       ...  
415    47482
416    47577
417    47698
418    47780
419    47782
Name: index, Length: 420, dtype: int64

In [10]:
index = isin[isin['Header']==True].reset_index()['index']
index
header = header.loc[index,:].reset_index()#[['Date','Header']]

In [11]:
header


,index,Date,Header
0,486,2015-07-1,BHP and FMG defy Rio's iron slump to beat expo...
1,625,2015-07-0,MUDRA Bill Likely to be in Parliament in Winte...
2,690,2015-07-2,CBS launches Small Business Pulse
3,756,2015-07-3,"Zaldivar not on menu for BHP, Rio"
4,897,2015-07-0,Corporation Bank launches MUDRA Card based on ...
...,...,...,...
415,47482,2015-10-2,REUTERS BUSINESS NEWS SCHEDULE AT 1830 GMT/2:3...
416,47577,2015-10-0,"Company Shares of S&T Bancorp, Inc. (NASDAQ:ST..."
417,47698,2015-10-2,"BHP Sept qtr iron ore output up 7 pct, sticks ..."
418,47780,2015-10-1,US stocks move lower in afternoon trading; Oil...


In [15]:
# Load "en_core_web_sm" NLP. To download, first pip install spacy, then 
# enter in a terminal "python3 -m spacy download en_core_web_sm"
nlp = spacy.load('en_core_web_sm')

#header = 'Goldman Sachs announce purchase of Microsoft'

# Enter your username in here. I've put the paths you wrote as an attribute in the dictionary below
user = 'Francesco'

# Dictionary containing the Java paths for each user
java_paths = {
    'Conor': '/usr/lib/jvm/java-8-openjdk-amd64',
    'Francesco': '/Users/macbookpro/Downloads/jdk-13.0.1.jdk/Contents/Home/bin/java',
    'Cecilia': 'C:/Program Files/Java/jdk-13.0.1/bin/java.exe'
}
os.environ['JAVAHOME'] = java_paths[user]

st = StanfordNERTagger(
    '/Users/macbookpro/Documents/GitHub/nlp/ner/stanford-ner-2014-06-16/classifiers/english.all.3class.distsim.crf.ser.gz',
    '/Users/macbookpro/Documents/GitHub/nlp/ner/stanford-ner-2014-06-16/stanford-ner.jar',
    encoding = 'utf-8'
)
COMPANY_TYPES = ['PERSON', 'ORGANIZATION']

# def get_ticker(name, tickers, max_words):
# 	'''Replace one (possible multiple-word) organisation name with its ticker
	
# 	Params:
# 	name (str): Company name (possibly multiple words)
# 	tickers (dict): Dictionary of (name, ticker) pairs
# 	max_words (number): The maximum number of words of any company name in tickers
# 	'''

# 	# Get the list of the words in the company name
# 	company_words = name.split(' ')

# 	for word_count in reversed(range(1, min(max_words, len(company_words))+1)):
# 		for index in range(0, len(company_words) - word_count + 1):
# 			subset = ' '.join(company_words[index : index+word_count])
# 			for company, ticker in tickers.items():
# 				# If this phrase is a substring of a company name, return that 
# 				if subset.lower() in company.lower():
# 					return ticker

# 				# If this is already a ticker, return that
# 				if subset.lower() == ticker.lower():
# 					return ticker

# 	return None

def get_ticker(name, tickers):
    for key, vol in ticker.items():
        if vol.upper() == name.upper():
            return f'__{vol}'
        if name.upper() in key.upper():
            return f'__{vol}'
    return None

# def parse_header(header, max_company_name_length = max((len(x) for x in tickers))):
def parse_header(header):
    '''Attempt to replace all organisations in a header with their ticker'''
    try:
        tokens = nltk.word_tokenize(header)
    except:
        return False
    try:
        classified_text = st.tag(tokens)
    except OSError:
        return

    parsed_words = []

    # If multiple words in a row are organisations, we want to group
    # these together and check if they are tickers
    i = 0
    while i < len(classified_text):
        if classified_text[i][1] in COMPANY_TYPES:
            # Get the list of consecutive words until the next word is not
            # of type 'ORGANIZATION'
            words_to_classify = [classified_text[i][0]]
            index = 1
            while True:
            # print(i + index)
            # print(classified_text[i + index])

                if i + index >= len(classified_text):
                    break
                if classified_text[i + index][1] not in COMPANY_TYPES:
                    break
                    
            words_to_classify.append(classified_text[i + index][0])
            index += 1

            complete_organisation = [' '.join(words_to_classify), 'ORGANIZATION']
            parsed_words.append(complete_organisation)
            i += index

        else:
            parsed_words.append(list(classified_text[i]))
            i += 1

    for i in range(len(parsed_words)):
        word, classification = tuple(parsed_words[i])

        if classification == 'ORGANIZATION':
#             ticker = get_ticker(word, tickers, max_company_name_length)
            ticker = get_ticker(word, tickers)            
            if ticker is not None:
                parsed_words[i][0] = f'__{ticker}'

    return ' '.join(x[0] for x in parsed_words)

# tests = [
# 	'Goldman Sachs announce purchase of Microsoft',
# 	'Goldman announces purchase of Microsoft',
# 	'Goldman Sachs announce purchase of Goldman Sachs',
# 	'GS announces purchase of Goldman Sachs',
# 	'gs announces purchase of Goldman',
# 	'GS announces purchase of Goldman',
# ]

# for test in tests:
# 	output = parse_header(test)
# 	print(f'\ninput: {test}')
# 	print(f'output: {output}')

# tests = [
# 	'Goldman Sachs',
# 	'Goldman',
# 	'Microsoft',
# 	'Google',
# 	'Goldman Sachs Goldman',
# 	'goldman',
# 	'HARTFORD FINANCIAL SERVICES',
# 	'HARTFORD FINANCIAL',
# 	'HARTFORD',
# ]

# for test in tests:
# 	print('\n{}'.format(test))
# 	print(parse_organisation(test, tickers, max_company_name_length))

In [16]:
nlp = spacy.load('en_core_web_sm')
example = nlp('gs announces purchase of Goldman')

extraction = list(textacy.extract.subject_verb_object_triples(example))

print('extraction', extraction)

extraction [(gs, announces, purchase)]


In [ ]:
print(len(header['Header']))
for i in range (len(header['Header'])):
    print(i)
    headline = header.loc[i,'Header']
    #print(headline)
    try : 
        header.loc[i,'with ticker']=parse_header(headline)
    except : 
        print("An exception occurred")
    
header

420
0
BHP and FMG defy Rio's iron slump to beat export guidance
An exception occurred
1
MUDRA Bill Likely to be in Parliament in Winter Session: CEO, MUDRA Bank


In [17]:
print(len(header['with ticker']))
for i in range (len(header['with ticker'])):
    print(i)
    headline = header.loc[i,'with ticker']
    #print(headline)
    try : 
        nlp_headline = nlp(headline)
        for j in range(nlp_headline):
            header.loc[i,'nlp' + str(j)]=nlp_headline[j]
            header[['nlp'+str(j)+'Sub','nlp'+str(j)+'Verb', 'nlp'+str(j)+'Obj']] = pd.DataFrame(header['nlp'+str(j)].tolist(), index=df.index)
    except : 
        print("An exception occurred")
    
header

AttributeError: 'Series' object has no attribute 'upper'

In [35]:
header

,index,Date,Header,with ticker
0,486,2015-07-1,BHP and FMG defy Rio's iron slump to beat expo...,BHP and FMG defy __MAR 's iron slump to beat e...
1,625,2015-07-0,MUDRA Bill Likely to be in Parliament in Winte...,MUDRA Bill Likely to be in Parliament in Winte...
2,690,2015-07-2,CBS launches Small Business Pulse,__CBS launches Small Business Pulse
3,756,2015-07-3,"Zaldivar not on menu for BHP, Rio","Zaldivar not on menu for BHP , Rio"
4,897,2015-07-0,Corporation Bank launches MUDRA Card based on ...,__PKG launches MUDRA Card based on Rupay platform
...,...,...,...,...
415,47482,2015-10-2,REUTERS BUSINESS NEWS SCHEDULE AT 1830 GMT/2:3...,REUTERS BUSINESS NEWS SCHEDULE AT 1830 GMT2:30...
416,47577,2015-10-0,"Company Shares of S&T Bancorp, Inc. (NASDAQ:ST...",__APD ( NASDAQ : STBA ) Drops by -2.05 %
417,47698,2015-10-2,"BHP Sept qtr iron ore output up 7 pct, sticks ...","BHP Sept qtr iron ore output up 7 pct , sticks..."
418,47780,2015-10-1,US stocks move lower in afternoon trading; Oil...,US stocks move lower in afternoon trading ; Oi...


In [42]:
header['nlp'] = header['with ticker'].apply(nlp)

#extraction = list(textacy.extract.subject_verb_object_triples(example))

TypeError: object of type 'float' has no len()